<a href="https://colab.research.google.com/github/stan-dev/example-models/blob/feature%2Fjupyter-notebooks/jupyter/covid-inf-rate/SantaClara_CmdStanR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Bayesian analysis inference of coronavirus infection rate from the Stanford study in Santa Clara county

This Google Colab Python notebook provides the models and data discussed here:  [Simple Bayesian analysis inference of coronavirus infection rate from the Stanford study in Santa Clara county](https://statmodeling.stat.columbia.edu/2020/05/01/simple-bayesian-analysis-inference-of-coronavirus-infection-rate-from-the-stanford-study-in-santa-clara-county/)

It uses CmdStanR to compile and fit the model.

#### First up, a little admin to get the latest CmdStanR and CmdStan installed on this instance.

In [0]:
# Preliminary setup
install.packages('versions')
library(versions)
# Install package CmdStanR from GitHub
library(devtools)
if(!require(cmdstanr)){
  devtools::install_github("stan-dev/cmdstanr", dependencies=c("Depends", "Imports"))
  library(cmdstanr)
}

In [0]:
# Install CmdStan binaries
if (!file.exists("cmdstan-2-22-1.tgz")) {
  system("wget https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz", intern=T)
  system("tar zxf colab-cmdstan-2.23.0.tar.gz", intern=T)
}
list.files("cmdstan-2.23.0")

In [0]:
# Set cmdstan_path to CmdStan installation
set_cmdstan_path("cmdstan-2.23.0")

In [0]:
# helper function
print_file <- function(file, nlines=-1L) {
  cat(paste(readLines(file, n=nlines), "\n", sep=""), sep="")
}

#### Upload models and data from GitHub

The models and data are available on the Stan GitHub repo https://github.com/stan-dev/example-models/tree/feature/jupyter-notebooks/jupyter/covid-inf-rate

In [0]:
system("wget https://raw.githubusercontent.com/stan-dev/example-models/feature/jupyter-notebooks/jupyter/covid-inf-rate/data/santa_clara_all.data.json", intern=T)
system("wget https://raw.githubusercontent.com/stan-dev/example-models/feature/jupyter-notebooks/jupyter/covid-inf-rate/data/santa_clara_apr_11.data.json", intern=T)
system("wget https://raw.githubusercontent.com/stan-dev/example-models/feature/jupyter-notebooks/jupyter/covid-inf-rate/data/santa_clara_apr_27.data.json", intern=T)
system("wget https://raw.githubusercontent.com/stan-dev/example-models/feature/jupyter-notebooks/jupyter/covid-inf-rate/stan/pool_sens_spec.stan", intern=T)
system("wget https://raw.githubusercontent.com/stan-dev/example-models/feature/jupyter-notebooks/jupyter/covid-inf-rate/stan/hier_sens_spec.stan", intern=T)
system("wget https://raw.githubusercontent.com/stan-dev/example-models/feature/jupyter-notebooks/jupyter/covid-inf-rate/stan/hier_sens_spec_offset_mult.stan", intern=T)
list.files()



In the Santa Clara study, of the 3330 people tested (`n_sample`), 50 test results were positive (`y_sample`).  The first version of the paper was released on April 11th, and used specificity data from two studies and sensitivity data from two other studies.


In [0]:
print_file("santa_clara_apr_11.data.json")

### Simple Model

The first Stan model in this blogpost is called "santaclara.stan".  It a complete pooling model, therefore we'll call it "pool_sens_spec.stan".

In [0]:
print_file("pool_sens_spec.stan")

We use CmdStanR to compile and fit the model to the data.

In [0]:
pool_model <- cmdstan_model(stan_file='pool_sens_spec.stan')
apr_11_fit <- pool_model$sample(data='santa_clara_apr_11.data.json')


In [0]:
options(digits = 2)
apr_11_fit$summary()


Given the April 11th dataset, the central 95% interval is [0.002 - 0.017], thus the data are consistent with an underlying infection rate of between 0 and 2%.

In [0]:
apr_11_draws <- apr_11_fit$draws()
hist(apr_11_draws[,,"p"])

We repeat this procedure with the data from the April 27th version of the study, which has specificity data from 13 studies and sensitivity data from 3 studies.  Again, we pool the results from all studies.

In [0]:
apr_27_fit <- pool_model$sample(data='santa_clara_apr_27.data.json')


In [0]:
apr_27_fit$summary()
apr_27_draws <- apr_27_fit$draws()
hist(apr_27_draws[,,"p"])


### Hierarchical Model

The hierarchical model allows the sensitivities and specificities to vary across studies.
In the blogpost is called "santaclara_hier.stan".  We'll call it "hier_sens_spec.stan



In [0]:
print_file("hier_sens_spec.stan")

__Note__: this model has the suggested tight prior on `sigma_sens` in order to compensate for only having data from 3 studies to fit.

For this model, the data is broken out by study.  We copied this data out of page 19 of the report and munged it into the forms required by the model's data block variable definitions.

In [0]:
print_file("santa_clara_all.data.json")

In [0]:
hier_model <- cmdstan_model(stan_file='hier_sens_spec.stan')
hier_fit <- hier_model$sample(data='santa_clara_all.data.json')


This central 95% interval is [0.004 - 0.022].

In [0]:
hier_fit$summary()
hier_draws <- hier_fit$draws()
hist(hier_draws[,,"p"])


Bob Carpenter provided a second version of the hierarchical model in which the parameters for the item-level specificities and sensitivies are specified using the Stan language "offset, multiplier" syntax which allows for affine transforms on real-valued variables.  See the Stan Language Reference Manual, chapter [Univariate Data Types and Variable Declarations](https://mc-stan.org/docs/2_23/reference-manual/univariate-data-types-and-variable-declarations.html) for further details. We call this model "hier_sens_spec_offset_mult.stan

In [0]:
print_file("hier_sens_spec_offset_mult.stan")

In [0]:
hier_model_v2 <- cmdstan_model(stan_file='hier_sens_spec_offset_mult.stan')
hier_fit_v2 <- hier_model_v2$sample(data='santa_clara_all.data.json')

In [0]:
hier_fit_v2$summary()
hier_draws_v2 <- hier_fit_v2$draws()
hist(hier_draws_v2[,,"p"])